In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.4 MB/s eta 0:00:00


In [2]:
# This code demonstrates GPT-2's language task ability
import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, pipeline, set_seed

# Please input the prompt you like
prompt = "Hello, I'm a language model," # TODO: replace with raiseError
generator = pipeline('text-generation', model='gpt2')
generated_result = generator(prompt, max_length=50, num_return_sequences=3)

for i, result in enumerate(generated_result):
    print("The {}th result begins:\n{}\nThe {}th result ends\n".format(i,result["generated_text"],i))

# Despite using pipeline directly provided by transformers, it is also good to take a closer look
prompt = "My dog is cute" # TODO: replace this with RaiseError 
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# tokens is a dictionary of multiple GPT2 parameters 
tokens = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **tokens,
    max_length=50
)

# Convert the outputs into words
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The 0th result begins:
Hello, I'm a language model, it means that I can be a functional model and I'm able to do things that are not true when we need to do so, that's important for this, that's going to be very important for understanding
The 0th result ends

The 1th result begins:
Hello, I'm a language model, but it takes more than just syntax. To do this for my project, I had to do various things, like set up my own documentation and set up a language for me. This took about a year from
The 1th result ends

The 2th result begins:
Hello, I'm a language model, using the standard syntax of the previous tutorial. Because some of you may know my work on VE is well documented, I couldn't imagine getting people to stop using VE.

Instead, I'm
The 2th result ends



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


My dog is cute and I love her. She is a great dog and I am very happy with her. I am very happy with my dog and she is a great dog.

I am a big fan of the dog and she is a


In [3]:
import requests
import tarfile

url = "https://drive.google.com/uc?id=0Bz8a_Dbh9QhbUDNpeUdjb0wxRms&export=download"
filename = "ag_news_csv.tar.gz"
extract_dir = "path/to/extract/dir"

# Download the dataset
response = requests.get(url, stream=True)
with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

In [4]:
   !tar -xzvf "/content/ag_news_csv.tar.gz" -C "/content/"

ag_news_csv/
ag_news_csv/train.csv
ag_news_csv/test.csv
ag_news_csv/classes.txt
ag_news_csv/readme.txt


In [18]:
!head -n 5000 /content/ag_news_csv/train.csv > /content/ag_news_csv/train_subset.csv
!head -n 10 /content/ag_news_csv/test.csv > /content/ag_news_csv/test_subset.csv

In [19]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

device = torch.device("cuda")
model.to(device)

# Load the AG News dataset
train_dataset = TextDataset(tokenizer=tokenizer, file_path="/content/ag_news_csv/train_subset.csv", block_size=128)
eval_dataset = TextDataset(tokenizer=tokenizer, file_path="/content/ag_news_csv/test_subset.csv", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="path/to/output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=500,
    warmup_steps=500,
    learning_rate=2e-5,
    fp16=True,
    logging_dir="path/to/logs",
    logging_steps=100,
    dataloader_num_workers=4,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)
trainer.train()

Step,Training Loss
100,4.481500
200,3.967100


TrainOutput(global_step=294, training_loss=4.053562086455676, metrics={'train_runtime': 49.1223, 'train_samples_per_second': 47.799, 'train_steps_per_second': 5.985, 'total_flos': 153378422784000.0, 'train_loss': 4.053562086455676, 'epoch': 1.0})

In [21]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# # Get the predictions and labels
# predictions = trainer.predict(eval_dataset)
# predicted_text = [tokenizer.decode(p, skip_special_tokens=True) for p in predictions.predictions]
# labels = [tokenizer.decode(l, skip_special_tokens=True) for l in predictions.label_ids]

# # Print the predicted text and labels
# for i in range(len(predicted_text)):
#     print(f"Text: {predicted_text[i]}")
#     print(f"Label: {labels[i]}")

eval_loss: 3.4756
eval_runtime: 3.3228
eval_samples_per_second: 137.8370
eval_steps_per_second: 17.4550
epoch: 1.0000
